In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##압축 해제

In [2]:

!apt-get install -y p7zip-full

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
!7z x "/content/drive/MyDrive/OpenSource/Data/amount_img.zip.part0" -o"/content/drive/MyDrive/OpenSource/Data/amount_img" -y


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/OpenSource/Data/                                                 1 file, 1073741824 bytes (1024 MiB)

Extracting archive: /content/drive/MyDrive/OpenSource/Data/amount_img.zip.part0

ERRORS:
Unexpected end of archive

--
Path = /content/drive/MyDrive/OpenSource/Data/amount_img.zip.part0
Type = zip
ERRORS:
Unexpected end of archive
Physical Size = 1077507111

  0%      1% 5 - image/갈비탕/Q2/Q2.zip                                    2% 9 - image/갈비탕/Q4/Q4.zip                                   

In [ ]:
import os
import zipfile

base_dir = "/content/drive/MyDrive/OpenSource/Data/unzipped_amount/image"

# 음식 폴더 루프
for food in os.listdir(base_dir):
    food_path = os.path.join(base_dir, food)
    if os.path.isdir(food_path):
        # Q1 ~ Q5 폴더 탐색
        for q_folder in os.listdir(food_path):
            q_path = os.path.join(food_path, q_folder)
            if os.path.isdir(q_path):
                for file in os.listdir(q_path):
                    if file.endswith(".zip"):
                        zip_path = os.path.join(q_path, file)
                        print(f"📦 Extracting: {zip_path}")
                        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                            zip_ref.extractall(q_path)
                        os.remove(zip_path)
                        print(f"🗑️ Deleted: {zip_path}"

📦 Trying: /content/drive/MyDrive/OpenSource/Data/unzipped_amount/image/돈가스/Q3/Q3.zip
❌ Failed to extract: /content/drive/MyDrive/OpenSource/Data/unzipped_amount/image/돈가스/Q3/Q3.zip

ERRORS:
Unexpected end of archive

ERROR: Data Error : top_어육류정형_사각배달_돈가스_Q3_00011.JPG



압축 해제 과정에서 돈까스 Q4, Q5 삭제

## 모델 구축

In [12]:
!pip install timm

# 1. 라이브러리 임포트
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
import timm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 129.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [13]:
# 2. 커스텀 데이터셋 정의
class AmountDataset(Dataset):
    def __init__(self, image_root, txt_root, transform=None):
        self.samples = []
        self.transform = transform
        for food in os.listdir(image_root):
            food_img_path = os.path.join(image_root, food)
            food_txt_path = os.path.join(txt_root, food)
            if not os.path.isdir(food_img_path) or not os.path.isdir(food_txt_path):
                continue
            for qty in os.listdir(food_img_path):  # Q1~Q5
                img_dir = os.path.join(food_img_path, qty)
                txt_dir = os.path.join(food_txt_path, qty)
                if not os.path.isdir(img_dir) or not os.path.isdir(txt_dir):
                    continue
                # Q1 -> 0, Q5 -> 4
                if qty.upper().startswith('Q') and qty[1:].isdigit():
                    label = int(qty[1:]) - 1
                else:
                    continue
                for img_file in os.listdir(img_dir):
                    # 확장자 대소문자 구분 없이 처리
                    if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                        continue
                    img_path = os.path.join(img_dir, img_file)
                    base_name, _ = os.path.splitext(img_file)
                    txt_path = os.path.join(txt_dir, base_name + '.txt')
                    if os.path.exists(txt_path):
                        self.samples.append((img_path, txt_path, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, txt_path, label = self.samples[idx]
        image = Image.open(img_path).convert("RGB")
        # 첫 bbox만 사용하여 crop
        with open(txt_path, 'r') as f:
            line = f.readline().strip().split()
            _, cx, cy, w, h = map(float, line)
        W, H = image.size
        x1 = int((cx - w / 2) * W)
        y1 = int((cy - h / 2) * H)
        x2 = int((cx + w / 2) * W)
        y2 = int((cy + h / 2) * H)
        image = image.crop((x1, y1, x2, y2))
        if self.transform:
            image = self.transform(image)
        return image, label

In [14]:
# 3. 데이터 전처리 및 데이터셋 로드
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

data_root = '/content/drive/MyDrive/OpenSource/Data'
image_root = os.path.join(data_root, 'image')
txt_root   = os.path.join(data_root, 'txt')

dataset = AmountDataset(image_root, txt_root, transform=transform)


In [16]:
# 4. 학습/테스트셋 분리
indices = list(range(len(dataset)))
labels = [dataset.samples[i][2] for i in indices]
train_idx, test_idx = train_test_split(indices, test_size=0.2, stratify=labels, random_state=42)
train_dataset = Subset(dataset, train_idx)
test_dataset  = Subset(dataset, test_idx)

# 워커 수를 적절히 설정하여 DataLoader 경고 방지 (최대 2)
num_workers = min(2, os.cpu_count() if os.cpu_count() is not None else 1)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=num_workers)
test_loader  = DataLoader(test_dataset,  batch_size=32, shuffle=False, num_workers=num_workers)


In [17]:
# 5. 모델 정의
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = timm.create_model('efficientnet_b0', pretrained=True)
model.classifier = nn.Linear(model.classifier.in_features, 5)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [18]:
# 6. 학습 루프 및 진행 상황 출력
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total_samples = 0
    for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        preds = outputs.argmax(1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data).item()
        total_samples += inputs.size(0)
    epoch_loss = running_loss / total_samples
    epoch_acc  = running_corrects / total_samples
    print(f'[Epoch {epoch+1}/{num_epochs}] Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')


[Epoch 1/5] Train Loss: 1.6174 Acc: 0.2159


[Epoch 2/5] Train Loss: 1.3754 Acc: 0.6127


[Epoch 3/5] Train Loss: 1.1870 Acc: 0.7968


[Epoch 4/5] Train Loss: 1.0238 Acc: 0.8889


[Epoch 5/5] Train Loss: 0.8501 Acc: 0.9270


In [20]:
# 7. 테스트 성능 평가 함수
def evaluate(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = outputs.argmax(1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    print("\n📊 Test Set Results:")
    print(classification_report(all_labels, all_preds, digits=4))
    print("Confusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))

In [21]:
# 8. 모델 테스트
evaluate(model, test_loader)


📊 Test Set Results:
              precision    recall  f1-score   support

           0     0.3750    0.2000    0.2609        15
           1     0.3200    0.4706    0.3810        17
           2     0.2632    0.3125    0.2857        16
           3     0.3846    0.3333    0.3571        15
           4     0.5000    0.4375    0.4667        16

    accuracy                         0.3544        79
   macro avg     0.3686    0.3508    0.3503        79
weighted avg     0.3677    0.3544    0.3517        79

Confusion Matrix:
[[3 6 4 1 1]
 [1 8 4 2 2]
 [3 4 5 2 2]
 [1 3 4 5 2]
 [0 4 2 3 7]]


In [26]:
# 9. Colab에서 이미지 업로드 후 예측
from google.colab import files
uploaded = files.upload()
for filename in uploaded.keys():
    predict_amount(filename, model, transform)

Saving side_기타튀김_배달종이박스_감자튀김(스틱형)_Q2_00043.JPG to side_기타튀김_배달종이박스_감자튀김(스틱형)_Q2_00043.JPG
📷 side_기타튀김_배달종이박스_감자튀김(스틱형)_Q2_00043.JPG -> Predicted Amount: Q1


In [27]:
# 10. 모델 저장
torch.save(model.state_dict(), '/content/efficientnet_amount_model.pth')

## 2차 모델 구축

In [2]:
!pip install timm

# 1. 라이브러리 임포트
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
import timm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [3]:
# 2. 데이터 전처리: 데이터 증강(train) + 검증용(transform)
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [4]:
# 3. 커스텀 데이터셋 정의
class AmountDataset(Dataset):
    def __init__(self, samples, transform=None):
        self.samples = samples  # [(img_path, txt_path, label), ...]
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, txt_path, label = self.samples[idx]
        image = Image.open(img_path).convert("RGB")
        # 첫 bbox만 사용하여 crop
        with open(txt_path, 'r') as f:
            line = f.readline().strip().split()
            _, cx, cy, w, h = map(float, line)
        W, H = image.size
        x1 = int((cx - w / 2) * W)
        y1 = int((cy - h / 2) * H)
        x2 = int((cx + w / 2) * W)
        y2 = int((cy + h / 2) * H)
        image = image.crop((x1, y1, x2, y2))
        if self.transform:
            image = self.transform(image)
        return image, label

In [5]:
# 4. 샘플 리스트 생성 (파일 경로 순회)
data_root = '/content/drive/MyDrive/OpenSource/Data'
image_root = os.path.join(data_root, 'image')
txt_root = os.path.join(data_root, 'txt')
samples = []
for food in os.listdir(image_root):
    food_img_path = os.path.join(image_root, food)
    food_txt_path = os.path.join(txt_root, food)
    if not os.path.isdir(food_img_path) or not os.path.isdir(food_txt_path):
        continue
    for qty in os.listdir(food_img_path):  # Q1~Q5
        img_dir = os.path.join(food_img_path, qty)
        txt_dir = os.path.join(food_txt_path, qty)
        if not os.path.isdir(img_dir) or not os.path.isdir(txt_dir):
            continue
        if qty.upper().startswith('Q') and qty[1:].isdigit():
            label = int(qty[1:]) - 1
        else:
            continue
        for img_file in os.listdir(img_dir):
            if not img_file.lower().endswith(('.jpg', '.jpeg', '.png', '.JPG')):
                continue
            img_path = os.path.join(img_dir, img_file)
            base_name, _ = os.path.splitext(img_file)
            txt_path = os.path.join(txt_dir, base_name + '.txt')
            if os.path.exists(txt_path):
                samples.append((img_path, txt_path, label))

In [6]:
# 5. 학습/검증 분리
indices = list(range(len(samples)))
labels = [samples[i][2] for i in indices]
train_idx, val_idx = train_test_split(indices, test_size=0.2,
                                      stratify=labels, random_state=42)
train_samples = [samples[i] for i in train_idx]
val_samples   = [samples[i] for i in val_idx]

# 6. Dataset 및 DataLoader 생성
train_dataset = AmountDataset(train_samples, transform=train_transform)
val_dataset   = AmountDataset(val_samples,   transform=val_transform)

num_workers = min(2, os.cpu_count() or 1)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,  num_workers=num_workers)
val_loader   = DataLoader(val_dataset,   batch_size=32, shuffle=False, num_workers=num_workers)


In [7]:
# 7. 모델 정의 + 전체 레이어 파인튜닝
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = timm.create_model('efficientnet_b0', pretrained=True)
# Ensure all parameters are trainable
for param in model.parameters():
    param.requires_grad = True
model.classifier = nn.Linear(model.classifier.in_features, 5)
model = model.to(device)

# 8. 손실, 옵티마이저, 스케줄러 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

In [8]:
# 9. 학습 루프 + 스케줄러
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = running_corrects = total_samples = 0
    for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        preds = outputs.argmax(1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data).item()
        total_samples += inputs.size(0)
    epoch_loss = running_loss / total_samples
    epoch_acc = running_corrects / total_samples
    scheduler.step()
    print(f'[Epoch {epoch+1}] Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')


[Epoch 1] Loss: 1.6077 Acc: 0.2063


[Epoch 2] Loss: 1.5543 Acc: 0.3079


[Epoch 3] Loss: 1.5255 Acc: 0.3873


[Epoch 4] Loss: 1.4723 Acc: 0.4381


[Epoch 5] Loss: 1.4532 Acc: 0.4413


[Epoch 6] Loss: 1.3981 Acc: 0.4889


[Epoch 7] Loss: 1.3827 Acc: 0.5397


[Epoch 8] Loss: 1.3684 Acc: 0.5397


[Epoch 9] Loss: 1.3590 Acc: 0.5492


[Epoch 10] Loss: 1.3524 Acc: 0.5524


In [ ]:
# 10. 검증 성능 평가
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        preds = outputs.argmax(1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
print("\n📊 Validation Results:")
print(classification_report(all_labels, all_preds, digits=4))
print("Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))
